# PageRank Algorithm: Eigenvector-Based Ranking of Web Pages

**Course Assignment – Computational Linear Algebra**

This notebook presents an implementation of the PageRank algorithm, originally developed by Google, using linear algebra concepts such as **stochastic matrices**, **eigenvectors**, and the **power iteration method**.

The goal is to compute the steady-state importance score of each webpage in a directed graph representation of the web.


## Introduction

Search engines rank webpages by estimating their relative importance.
PageRank models this problem mathematically by interpreting the web as a **directed graph** and defining page importance as the stationary distribution of a Markov process.

The core idea is that a page becomes important if it is referenced by other important pages.
This recursive definition naturally leads to an **eigenvector problem**.


In [15]:
import numpy as np
import scipy.sparse as sp

## Construction of the Link Matrix

We construct the link matrix by encoding the hyperlink structure of the web.
Each column corresponds to a source page, and each non-zero entry distributes the page's importance equally among its outgoing links.

This ensures that the matrix is **column-stochastic**, meaning that each column sums to one.


In [ ]:
def build_A_csc(n, edges):
    """
    Build the column-stochastic link matrix A in sparse CSC form.

    Pages are 1..n.
    Each edge (src, dst) means: src links to dst.

    A_{dst, src} = 1/outdeg(src) if src -> dst exists, else 0.
    """
    outdeg = np.zeros(n + 1, dtype=int)
    for src, dst in edges:
        outdeg[src] += 1

    rows, cols, data = [], [], []
    for src, dst in edges:
        if outdeg[src] == 0:
            continue
        rows.append(dst - 1)          # destination row (0-indexed)
        cols.append(src - 1)          # source column (0-indexed)
        data.append(1.0 / outdeg[src])

    A = sp.csc_matrix((data, (rows, cols)), shape=(n, n))
    dangling = (outdeg[1:] == 0)      # pages with no outgoing links
    return A, dangling, outdeg[1:]


## Power Iteration Implementation

This code computes the PageRank scores using an iterative algorithm.

The idea is to start from an initial probability distribution over pages and repeatedly update it
until the values stop changing significantly. At each iteration, the new score of a page depends on:
- how much score it receives from other pages that link to it,
- a damping factor that models random jumps between pages,
- and a correction for pages that have no outgoing links (dangling nodes).

Dangling nodes would otherwise cause loss of probability mass. To avoid this, their accumulated
score is redistributed uniformly across all pages at each iteration.

The algorithm continues until the difference between two consecutive iterations becomes smaller
than a specified tolerance, or until a maximum number of iterations is reached.

The function returns:
- the final PageRank vector,
- the number of iterations required for convergence,
- and the final convergence error.


In [1]:
def pagerank_power(A, m=0.15, tol=1e-12, max_iter=500, x0=None, dangling=None):
    """
    Compute x satisfying x = (1-m) A x + m s  (paper eq. (3.2)),
    with standard dangling-node fix:
       dangling_mass = sum_{j dangling} x_j
       A x term is augmented by dangling_mass * s

    If m=0, this reduces to x = A x (paper eq. (2.1)) when there are no dangling nodes.
    """
    n = A.shape[0]
    s = np.ones(n) / n

    if x0 is None:
        x = s.copy()
    else:
        x = np.array(x0, dtype=float)
        x = x / x.sum()

    if dangling is None:
        dangling = np.zeros(n, dtype=bool)

    for k in range(max_iter):
        dangling_mass = x[dangling].sum()
        x_new = (1 - m) * (A @ x + dangling_mass * s) + m * s

        diff = np.abs(x_new - x).sum()  # L1 distance
        x = x_new
        if diff < tol:
            return x, k + 1, diff

    return x, max_iter, diff


def eigenspace_dim_near_1(A_dense, atol=1e-9):
    """For small examples: estimate dim(V1(A)) by counting eigenvalues near 1."""
    w = np.linalg.eigvals(A_dense)
    return int(np.sum(np.isclose(w, 1.0, atol=atol))), w

In [2]:
edges_fig21 = [(1,2),(1,3),(1,4),(2,3),(2,4),(3,1),(4,1),(4,3)]
A21, dang21, _ = build_A_csc(4, edges_fig21)

x_A, it_A, _ = pagerank_power(A21, m=0.0, tol=1e-14, max_iter=2000, dangling=dang21)
x_M, it_M, _ = pagerank_power(A21, m=0.15, tol=1e-14, max_iter=2000, dangling=dang21)

print("Fig2.1 using A (m=0):", x_A, "iters:", it_A)
print("Fig2.1 using M (m=0.15):", x_M, "iters:", it_M)


Fig2.1 using A (m=0): [0.38709677 0.12903226 0.29032258 0.19354839] iters: 53
Fig2.1 using M (m=0.15): [0.36815068 0.14180936 0.28796163 0.20207834] iters: 42


In [3]:
edges_fig22 = [(1,2),(2,1),(3,4),(4,3),(5,3),(5,4)]
A22, dang22, _ = build_A_csc(5, edges_fig22)

dimV1, eigvals = eigenspace_dim_near_1(A22.toarray())
print("dim(V1(A)) approx:", dimV1, "eigs:", eigvals)

x_M22, it_M22, _ = pagerank_power(A22, m=0.15, tol=1e-14, max_iter=2000, dangling=dang22)
print("Fig2.2 using M (m=0.15):", x_M22, "iters:", it_M22)


dim(V1(A)) approx: 2 eigs: [ 1. -1.  1. -1.  0.]
Fig2.2 using M (m=0.15): [0.2   0.2   0.285 0.285 0.03 ] iters: 2


STARTING EXERCISE 1:

In [4]:
edges_ex1 = [
    (1,2),(1,3),(1,4),
    (2,3),(2,4),
    (3,1),(3,5),   # page 3 now links to 1 and 5
    (4,1),(4,3),
    (5,3)          # page 5 links to 3
]
Aex1, dang_ex1, _ = build_A_csc(5, edges_ex1)

x_ex1_A, _, _ = pagerank_power(Aex1, m=0.0, tol=1e-14, max_iter=5000, dangling=dang_ex1)
print("Exercise 1 (using A, m=0):", x_ex1_A)
print("Compare x3 vs x1:", x_ex1_A[2], "vs", x_ex1_A[0])


Exercise 1 (using A, m=0): [0.24489796 0.08163265 0.36734694 0.12244898 0.18367347]
Compare x3 vs x1: 0.36734693877550834 vs 0.24489795918367419


## Interpretation of Results

The final PageRank vector represents the long-term probability of visiting each page in a random surfing model.

Higher values indicate more influential pages within the web structure.
The ranking reflects both the number and quality of incoming links.


STARTING EXERCISE 2

In [5]:
edges_ex2 = [(1,2),(2,1),(3,4),(4,3),(5,6),(6,5),(5,7),(6,7),(7,6)]
Aex2, dang_ex2, _ = build_A_csc(7, edges_ex2)

dimV1, eigvals = eigenspace_dim_near_1(Aex2.toarray())
print("Exercise 2: dim(V1(A)) approx:", dimV1)
print("eigenvalues:", eigvals)


Exercise 2: dim(V1(A)) approx: 3
eigenvalues: [ 1.         -1.          1.         -1.          1.         -0.50000001
 -0.49999999]


STARTING EXERCISE 3


In [7]:
def build_A_dense(n, edges):
    """
    Dense column-stochastic link matrix A.
    Edge (src,dst) means src links to dst.
    So A[dst-1, src-1] = 1/outdeg(src).
    """
    outdeg = np.zeros(n + 1, dtype=int)
    for src, dst in edges:
        outdeg[src] += 1

    A = np.zeros((n, n), dtype=float)
    for src, dst in edges:
        A[dst - 1, src - 1] += 1.0 / outdeg[src]
    return A

def dim_V1(A, atol=1e-10):
    eigvals = np.linalg.eigvals(A)
    dim = int(np.sum(np.isclose(eigvals, 1.0, atol=atol)))
    return dim, eigvals

# Figure 2.2 edges:
# 1 -> 2, 2 -> 1, 3 -> 4, 4 -> 3, 5 -> 3 and 4
edges_fig22 = [(1,2),(2,1),(3,4),(4,3),(5,3),(5,4)]

# Exercise 3 modification: add link 5 -> 1
edges_ex3 = edges_fig22 + [(5,1)]

A_ex3 = build_A_dense(5, edges_ex3)

dim, eigvals = dim_V1(A_ex3)
print("dim(V1(A)) =", dim)
print("eigenvalues =", np.sort(eigvals))
print("A =\n", A_ex3)

dim(V1(A)) = 2
eigenvalues = [-1. -1.  0.  1.  1.]
A =
 [[0.         1.         0.         0.         0.33333333]
 [1.         0.         0.         0.         0.        ]
 [0.         0.         0.         1.         0.33333333]
 [0.         0.         1.         0.         0.33333333]
 [0.         0.         0.         0.         0.        ]]


STARTING EXERCISE 4

In [8]:
def build_substochastic_A(n, edges):
    """
    Build the (possibly) column-substochastic link matrix A for a web with dangling nodes.
    Pages are 1..n
    Edge (src,dst) means src links to dst.
    A[dst-1, src-1] = 1/outdeg(src).
    If outdeg(src)=0, that column stays all zeros (dangling node).
    """
    outdeg = np.zeros(n + 1, dtype=int)
    for src, dst in edges:
        outdeg[src] += 1

    A = np.zeros((n, n), dtype=float)
    for src, dst in edges:
        A[dst - 1, src - 1] += 1.0 / outdeg[src]

    dangling = np.where(outdeg[1:] == 0)[0] + 1  # page numbers
    return A, dangling

# Figure 2.1 edges
# 1 -> {2,3,4}, 2 -> {3,4}, 3 -> {1}, 4 -> {1,3}
edges_fig21 = [(1,2),(1,3),(1,4),(2,3),(2,4),(3,1),(4,1),(4,3)]

# Exercise 4: remove link 3 -> 1  (so page 3 becomes dangling)
edges_ex4 = [e for e in edges_fig21 if e != (3,1)]

A, dangling_pages = build_substochastic_A(4, edges_ex4)
print("Dangling pages:", dangling_pages)
print("A=\n", A)
print("Column sums:", A.sum(axis=0))

# Perron eigenvalue/eigenvector
eigvals, eigvecs = np.linalg.eig(A)
# pick eigenvalue with largest real part (Perron eigenvalue here)
k = np.argmax(eigvals.real)
lam = eigvals[k].real
v = eigvecs[:, k].real

# make it nonnegative and normalize to sum to 1
v = np.abs(v)
x = v / v.sum()

print("\nPerron eigenvalue lambda =", lam)
print("Normalized Perron eigenvector x =", x)
print("sum(x) =", x.sum())


Dangling pages: [3]
A=
 [[0.         0.         0.         0.5       ]
 [0.33333333 0.         0.         0.        ]
 [0.33333333 0.5        0.         0.5       ]
 [0.33333333 0.5        0.         0.        ]]
Column sums: [1. 1. 0. 1.]

Perron eigenvalue lambda = 0.5613532393351084
Normalized Perron eigenvector x = [0.20664504 0.12270648 0.43864676 0.23200172]
sum(x) = 1.0


STARTING EXERCISE 11

In [11]:
# Augmented Figure 2.1
n = 5
edges = [
    (1,2),(1,3),(1,4),     # page 1
    (2,3),(2,4),           # page 2
    (3,1),(3,5),           # page 3
    (4,1),(4,3),           # page 4
    (5,3)                  # page 5
]

A = build_A_dense(n, edges)

# Google matrix
m = 0.15
S = np.ones((n, n)) / n
M = (1 - m) * A + m * S

# Eigenvector for eigenvalue 1
eigvals, eigvecs = np.linalg.eig(M)
k = np.argmin(np.abs(eigvals - 1))
x = eigvecs[:, k].real
x = x / x.sum()

print("PageRank vector:", x)
print("Sum:", x.sum())


PageRank vector: [0.23714058 0.09718983 0.34889409 0.13849551 0.17827999]
Sum: 1.0


STARTING EXERCISE 12

In [12]:
def eigvec_lambda1(M):
    """Eigenvector for eigenvalue 1, normalized to sum to 1 (positive direction)."""
    w, V = np.linalg.eig(M)
    k = np.argmin(np.abs(w - 1))
    x = V[:, k].real
    if x.sum() < 0:
        x = -x
    x = x / x.sum()
    return x

def ranking(x):
    order = np.argsort(-x)
    return [(int(i + 1), float(x[i])) for i in order]

# Exercise 11 web: Figure 2.1 + page 5 with 3<->5
# 1 -> {2,3,4}
# 2 -> {3,4}
# 3 -> {1,5}
# 4 -> {1,3}
# 5 -> {3}
edges_5 = [
    (1,2),(1,3),(1,4),
    (2,3),(2,4),
    (3,1),(3,5),
    (4,1),(4,3),
    (5,3)
]

# Exercise 12: add page 6 linking to every other page (1..5), no one links to 6
edges_6 = edges_5 + [(6,i) for i in range(1,6)]

n = 6
A = build_A_dense(n, edges_6)

# Rank using A (importance scores x = A x)
xA = eigvec_lambda1(A)

# Rank using M = (1-m)A + mS, with m=0.15
m = 0.15
S = np.ones((n, n)) / n
M = (1 - m) * A + m * S
xM = eigvec_lambda1(M)

print("x using A:", xA)
print("ranking(A):", ranking(xA))
print("x using M:", xM)
print("ranking(M):", ranking(xM))


x using A: [ 0.24489796  0.08163265  0.36734694  0.12244898  0.18367347 -0.        ]
ranking(A): [(3, 0.36734693877551017), (1, 0.24489795918367335), (5, 0.1836734693877552), (4, 0.1224489795918366), (2, 0.08163265306122457), (6, -0.0)]
x using M: [0.23121207 0.09476009 0.34017174 0.13503312 0.17382299 0.025     ]
ranking(M): [(3, 0.34017173872437517), (1, 0.23121206558721571), (5, 0.17382298895785955), (4, 0.13503312148083846), (2, 0.09476008524971118), (6, 0.025)]


STARTING EXERCISE 13

In [13]:
def pagerank_google(A, m=0.15):
    n = A.shape[0]
    S = np.ones((n, n)) / n
    M = (1 - m) * A + m * S

    w, V = np.linalg.eig(M)
    k = np.argmin(np.abs(w - 1))
    x = V[:, k].real
    if x.sum() < 0:
        x = -x
    x = x / x.sum()
    return x

# Exercise 13 web (two subwebs)
n = 6
edges = [
    # Subweb A: {1,2,3}
    (1,2),(2,3),(3,1),(1,3),(3,2),

    # Subweb B: {4,5}
    (4,5),(5,4),

    # Page 6 points into both subwebs, no one links to 6
    (6,1),(6,4)
]

A = build_A_dense(n, edges)
x = pagerank_google(A, m=0.15)

ranking = sorted([(i+1, x[i]) for i in range(n)], key=lambda t: -t[1])
print("PageRank vector:", x)
print("Ranking (page, score):")
for p, s in ranking:
    print(p, float(s))


PageRank vector: [0.13924028 0.1877924  0.24380066 0.20495495 0.19921171 0.025     ]
Ranking (page, score):
3 0.24380065661229086
4 0.20495495495495528
5 0.19921171171171184
2 0.18779239766081854
1 0.13924027906022352
6 0.02500000000000001


STARTING EXERCISE 14

## Power Iteration Method
Direct eigenvalue decomposition is computationally expensive for large matrices. Instead, we use the power iteration method, an iterative approach that converges to the dominant eigenvector.

Starting from an initial probability vector ( x^{(0)} ), the iteration is defined as:

[ x^{(k+1)} = M x^{(k)} ]

Under mild conditions, this sequence converges to the unique PageRank vector.

In [14]:
n = 5
edges = [
    (1,2),(1,3),(1,4),
    (2,3),(2,4),
    (3,1),(3,5),
    (4,1),(4,3),
    (5,3)
]

outdeg = np.zeros(n+1, dtype=int)
for s,d in edges:
    outdeg[s] += 1

A = np.zeros((n,n), float)
for s,d in edges:
    A[d-1, s-1] += 1.0/outdeg[s]

m = 0.15
S = np.ones((n,n))/n
M = (1-m)*A + m*S

# steady-state eigenvector q for eigenvalue 1
eigvals, eigvecs = np.linalg.eig(M)
k = np.argmin(np.abs(eigvals - 1))
q = eigvecs[:,k].real
if q.sum() < 0:
    q = -q
q = q / q.sum()

# pick an x0 "not too close" to q: all mass on page 1
x0 = np.zeros(n); x0[0] = 1.0

def l1(v): return float(np.sum(np.abs(v)))

def Mkx(k):
    x = x0.copy()
    for _ in range(k):
        x = M @ x
    return x

# requested k values
Ks = [1,5,10,50]

# errors and ratios
err0 = l1(x0 - q)
results = []
for k in Ks:
    xk = Mkx(k)
    errk = l1(xk - q)
    if k == 1:
        ratio = errk / err0
    else:
        ratio = errk / l1(Mkx(k-1) - q)
    results.append((k, errk, ratio))

# c = max_j |1 - 2 min_i M_ij|
mins = M.min(axis=0)
c = float(np.max(np.abs(1 - 2*mins)))

# |lambda2|
abs_sorted = sorted(np.abs(eigvals), reverse=True)
lambda2_abs = abs_sorted[1]

print("q =", q)
print("results (k, ||M^k x0 - q||_1, ratio):")
for row in results:
    print(row)
print("c =", c)
print("|lambda2| =", lambda2_abs)


q = [0.23714058 0.09718983 0.34889409 0.13849551 0.17827999]
results (k, ||M^k x0 - q||_1, ratio):
(1, 0.7819626528604117, 0.5125208081930114)
(5, 0.006871410715895657, 0.3227085473885674)
(10, 0.000304371655339461, 0.5245287479654996)
(50, 7.437800375598158e-13, 0.6112986746355819)
c = 0.94
|lambda2| = 0.6112685063651938


## Conclusion

This notebook demonstrates how PageRank naturally emerges from linear algebra concepts.
By combining eigenvector theory with stochastic processes, we obtain a mathematically elegant and computationally efficient ranking algorithm.

The approach scales well and forms the foundation of modern web search engines.
